In [1]:
#abrir arquivo lista_loja.xlsx
import pandas as pd
import openpyxl
import os
                           

In [2]:
#ler /Users/franciscoabraao/Documents/Jubart_2024/Painel_Precos_scrap/lista_lojas.xlsx
list_lojas = pd.read_excel('/Users/franciscoabraao/Documents/Jubart_2024/Painel_Precos_scrap/lista_lojas.xlsx')    

#ler /Users/franciscoabraao/Documents/Jubart_2024/Painel_Precos_scrap/lista_lojas.xlsx
df = pd.read_csv('/Users/franciscoabraao/Documents/Jubart_2024/Painel_Precos_scrap/precos_carrefour_kani.csv')  

In [3]:
#alterar a coluna preco de R$ 22,69 para 22.69 (float)
df['preco'] = df['preco'].str.replace('R$', '').str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df['preco'] = df['preco'].astype(float) 

#renomear coluna cidade_estado para cidade/UF do dataframe df
df.rename(columns={'cidade_estado': 'cidade/UF'}, inplace=True)  
#substituir " - " por "/" na coluna cidade/UF do dataframe df
df['cidade/UF'] = df['cidade/UF'].str.replace(' - ', '/')  
#separa o bairro de cidade/UF
df['bairro'] = df['cidade/UF'].str.split(',', expand=True)[0]
df['cidade/UF'] = df.apply(lambda row: row['cidade/UF'].replace(row['bairro'], '').strip(', '), axis=1)
# junta bairro a loja
df['loja'] = df['loja']+df['bairro']

#criando a coluna UF
df['UF'] = df['cidade/UF'].str[-2:]

# Mapeamento de estados para regiões
uf_to_region = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste',
    'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

# Criar a nova coluna com base no mapeamento
df['regiao'] = df['UF'].map(uf_to_region)

df = df[['data','produto', 'preco', 'cidade/UF', 'UF', 'regiao', 'loja', 'bairro']]

df

,data,produto,preco,cidade/UF,UF,regiao,loja,bairro
0,2025-04-18,Kani Congelado Frescatto 200g,13.39,Maceió/AL,AL,Nordeste,Hiper Maceió JatiúcaMangabeiras,Mangabeiras
1,2025-04-18,Kani Congelado Frescatto 200g,12.99,Itabuna/BA,BA,Nordeste,Big Bompreço ItabunaGoes Calmon,Goes Calmon
2,2025-04-18,Kani Congelado Frescatto 200g,12.59,Brasília/DF,DF,Centro-Oeste,Hipermercado Brasilia SulZona Industrial (Guará),Zona Industrial (Guará)
3,2025-04-18,Kani Congelado Frescatto 200g,12.19,Taguatinga/DF,DF,Centro-Oeste,Hipermercado Brasilia TaguatingaAguas Claras,Aguas Claras
4,2025-04-18,Kani Congelado Costa Sul 200g,13.39,Vila Velha/ES,ES,Sudeste,Hipermercado Vila Velha ShoppingDivino Espirit...,Divino Espirito Santo
...,...,...,...,...,...,...,...,...
91,2025-04-19,Kani Congelado Costa Sul 200g,10.19,Sorocaba/Votorantim/SP,SP,Sudeste,Hipermercado Sorocaba NorteVila Carvalho,Vila Carvalho
92,2025-04-19,Kani Congelado Costa Sul 200g,10.19,Taboão da Serra/SP,SP,Sudeste,Hipermercado TaboãoCentro,Centro
93,2025-04-19,Kani Congelado Costa Sul 200g,10.49,Taubate/SP,SP,Sudeste,Carrefour Bairro Taubaté IICentro,Centro
94,2025-04-19,Kani Congelado Costa Sul 200g,14.79,Vinhedo/SP,SP,Sudeste,Carrefour Bairro VinhedoVista Alegre,Vista Alegre


In [4]:
df_maps = df.merge(list_lojas[['lat', 'long','loja']], how='left', left_on='loja', right_on='loja')
df_maps  

,data,produto,preco,cidade/UF,UF,regiao,loja,bairro,lat,long
0,2025-04-18,Kani Congelado Frescatto 200g,13.39,Maceió/AL,AL,Nordeste,Hiper Maceió JatiúcaMangabeiras,Mangabeiras,-9.64959,-35.71739
1,2025-04-18,Kani Congelado Frescatto 200g,12.99,Itabuna/BA,BA,Nordeste,Big Bompreço ItabunaGoes Calmon,Goes Calmon,-14.79790,-39.27272
2,2025-04-18,Kani Congelado Frescatto 200g,12.59,Brasília/DF,DF,Centro-Oeste,Hipermercado Brasilia SulZona Industrial (Guará),Zona Industrial (Guará),-15.82938,-47.95478
3,2025-04-18,Kani Congelado Frescatto 200g,12.19,Taguatinga/DF,DF,Centro-Oeste,Hipermercado Brasilia TaguatingaAguas Claras,Aguas Claras,-15.82804,-48.04138
4,2025-04-18,Kani Congelado Costa Sul 200g,13.39,Vila Velha/ES,ES,Sudeste,Hipermercado Vila Velha ShoppingDivino Espirit...,Divino Espirito Santo,-20.34885,-40.29805
...,...,...,...,...,...,...,...,...,...,...
91,2025-04-19,Kani Congelado Costa Sul 200g,10.19,Sorocaba/Votorantim/SP,SP,Sudeste,Hipermercado Sorocaba NorteVila Carvalho,Vila Carvalho,NaN,NaN
92,2025-04-19,Kani Congelado Costa Sul 200g,10.19,Taboão da Serra/SP,SP,Sudeste,Hipermercado TaboãoCentro,Centro,NaN,NaN
93,2025-04-19,Kani Congelado Costa Sul 200g,10.49,Taubate/SP,SP,Sudeste,Carrefour Bairro Taubaté IICentro,Centro,NaN,NaN
94,2025-04-19,Kani Congelado Costa Sul 200g,14.79,Vinhedo/SP,SP,Sudeste,Carrefour Bairro VinhedoVista Alegre,Vista Alegre,NaN,NaN


In [40]:
import plotly.express as px

# Criar o gráfico de pontos
fig = px.scatter(
    df_maps,
    x="cidade/UF",
    y="preco",
    color="produto",
    title="Preços por Produto e Cidade",
    labels={"cidade/UF": "Cidade/UF", "preco": "Preço (R$)"},
    category_orders={"cidade/UF": sorted(df_maps["cidade/UF"].unique())}  # Ordenar as cidades
)

# Atualizar layout para separar as cidades por região
fig.update_layout(
    xaxis=dict(
        tickangle=45,
        title="Cidades (separadas por região)"
    ),
    yaxis=dict(title="Preço (R$)")
)

# Exibir o gráfico
fig.show()

In [42]:
import plotly.express as px

# Ordenar as cidades por região e dentro de cada região por ordem alfabética
df_maps['regiao_ordem'] = df_maps['regiao'] + ' - ' + df_maps['cidade/UF']
df_maps = df_maps.sort_values(by=['regiao_ordem', 'cidade/UF'])

# Criar o gráfico de pontos
fig = px.scatter(
    df_maps,
    x="regiao_ordem",
    y="preco",
    color="produto",
    title="Preços por Produto e Cidade (Separados por Região)",
    labels={"regiao_ordem": "Cidade/UF (por Região)", "preco": "Preço (R$)"},
)
# Atualizar layout para melhorar a visualização
fig.update_layout(
    xaxis=dict(
        tickangle=45,
        title="Cidades (separadas por região)"
    ),
    yaxis=dict(title="Preço (R$)")
)

# Exibir o gráfico
fig.show()

In [45]:
# Criar o gráfico de pontos interativo
fig = px.scatter(df,
                 x='cidade/UF',
                 y='preco',
                 color='produto',
                 facet_col='regiao',
                 facet_col_wrap=2,  # Organiza em 2 colunas
                 title='Preços de Produtos por Cidade/UF e Região',
                 labels={'preco': 'Preço (R$)', 'cidade/UF': 'Cidade/UF'},
                 height=800,
                 width=1000)

# Melhorar a formatação
fig.update_layout(
    hovermode='closest',
    xaxis_title='Cidade/UF',
    yaxis_title='Preço (R$)',
    legend_title='Produto'
)

# Rotacionar os rótulos do eixo X para melhor visualização
fig.update_xaxes(tickangle=45)

# Ajustar o espaçamento entre subplots
fig.update_layout(margin=dict(l=50, r=50, b=150, t=100))

# Mostrar o gráfico
fig.show()

In [46]:
regioes = df['regiao'].unique()

for regiao in regioes:
    df_reg = df[df['regiao'] == regiao]
    fig = px.scatter(df_reg,
                     x='cidade/UF',
                     y='preco',
                     color='produto',
                     title=f'Preços de Produtos - Região {regiao}',
                     labels={'preco': 'Preço (R$)', 'cidade/UF': 'Cidade/UF'},
                     height=600,
                     width=900)
    
    fig.update_layout(
        hovermode='closest',
        xaxis_title='Cidade/UF',
        yaxis_title='Preço (R$)',
        legend_title='Produto',
        margin=dict(l=50, r=50, b=150, t=100)
    )
    fig.update_xaxes(tickangle=45)
    fig.show()


In [48]:
df['produto'].unique()

array(['Kani Congelado Frescatto 200g', 'Kani Congelado Costa Sul 200g',
       'Kani Congelado Damm 250g',
       'Hossomaki Philadelphia Kani Kama 160 g'], dtype=object)

In [57]:
# Dicionário de cores fixas
cores_produtos = {
    'Kani Congelado Frescatto 200g': 'blue',
    'Kani Congelado Costa Sul 200g': 'red',
    'Kani Congelado Damm 250g': 'orange',
    'Hossomaki Philadelphia Kani Kama 160 g': 'green'
}


In [58]:
regioes = df['regiao'].unique()

for regiao in regioes:
    df_reg = df[df['regiao'] == regiao]
    
    fig = px.scatter(df_reg,
                     x='cidade/UF',
                     y='preco',
                     color='produto',
                     color_discrete_map=cores_produtos,
                     title=f'Preços de Produtos - Região {regiao}',
                     labels={'preco': 'Preço (R$)', 'cidade/UF': 'Cidade/UF'},
                     height=600,
                     width=900)
    
    fig.update_layout(
        hovermode='closest',
        xaxis_title='Cidade/UF',
        yaxis_title='Preço (R$)',
        legend_title='Produto',
        margin=dict(l=50, r=50, b=150, t=100)
    )
    fig.update_xaxes(tickangle=45)
    fig.show()


In [68]:
# Verificar se há valores NaN em todo o DataFrame
df.isna().any()

# Contar o número de valores NaN em cada coluna
df.isna().sum()

data            0
produto         0
preco           0
cidade/UF       0
UF              0
regiao          0
loja            0
bairro          0
lat             0
long            0
regiao_ordem    0
dtype: int64

In [72]:
#excluir dados NaN
df = df.dropna()

In [67]:
df

,data,produto,preco,cidade/UF,UF,regiao,loja,bairro,lat,long,regiao_ordem
0,2025-04-18,Kani Congelado Frescatto 200g,11.39,Anapolis/GO,GO,Centro-Oeste,Hipermercado Goiânia AnápolisCidade Jardim,Cidade Jardim,-16.32138,-48.94862,Centro-Oeste - Anapolis/GO
1,2025-04-18,Kani Congelado Frescatto 200g,12.59,Brasília/DF,DF,Centro-Oeste,Hipermercado Brasilia SulZona Industrial (Guará),Zona Industrial (Guará),-15.82938,-47.95478,Centro-Oeste - Brasília/DF
2,2025-04-18,Kani Congelado Costa Sul 200g,14.25,Campo Grande/MS,MS,Centro-Oeste,Hipermercado Campo GrandeSanta Fe,Santa Fe,-20.45765,-54.58963,Centro-Oeste - Campo Grande/MS
3,2025-04-18,Kani Congelado Frescatto 200g,12.45,Campo Grande/MS,MS,Centro-Oeste,Hipermercado Campo GrandeSanta Fe,Santa Fe,-20.45765,-54.58963,Centro-Oeste - Campo Grande/MS
4,2025-04-18,Kani Congelado Frescatto 200g,11.39,Goiania/GO,GO,Centro-Oeste,Hipermercado Goiânia SulJd Goias,Jd Goias,-16.70961,-49.23282,Centro-Oeste - Goiania/GO
...,...,...,...,...,...,...,...,...,...,...,...
91,2025-04-18,Kani Congelado Frescatto 200g,9.29,Porto Alegre/RS,RS,Sul,Hipermercado PartenonPartenon,Partenon,-30.05709,-51.16583,Sul - Porto Alegre/RS
92,2025-04-18,Kani Congelado Costa Sul 200g,10.99,Santa Maria/RS,RS,Sul,Hipermercado Santa MariaCentro,Centro,-29.67947,-53.80657,Sul - Santa Maria/RS
93,2025-04-18,Kani Congelado Frescatto 200g,9.89,Santa Maria/RS,RS,Sul,Hipermercado Santa MariaCentro,Centro,-29.67947,-53.80657,Sul - Santa Maria/RS
94,2025-04-18,Kani Congelado Costa Sul 200g,13.69,Viamão/RS,RS,Sul,BIG ViamãoCecília,Cecília,-30.07905,-51.10104,Sul - Viamão/RS


In [69]:
import pandas as pd
import folium
from branca.colormap import linear
from IPython.display import display

# Carregar os dados
#df = pd.read_csv('precos_carrefour_mapa.csv')

# Garantir que as colunas 'lat', 'long', 'preco' e 'loja' existam
# (ajuste os nomes conforme necessário)

# Criar o mapa centrado no Brasil
mapa = folium.Map(location=[-15.793889, -47.882778], zoom_start=4)

# Definir escala de cores para os preços
preco_minimo = df['preco'].min()
preco_maximo = df['preco'].max()
colormap = linear.YlOrRd_09.scale(preco_minimo, preco_maximo)

# Adicionar círculos com popup
for _, row in df.iterrows():
    circle = folium.CircleMarker(
        location=[row['lat'], row['long']],
        radius=10,
        color='black',
        fill=True,
        fill_opacity=0.7,
        fill_color=colormap(row['preco'])
    )
    
    # Popup com loja e preço
    popup_html = f"<b>{row['loja']}</b><br>{row['produto']}<br>Preço: <b>R${row['preco']:.2f}</b>"
    popup = folium.Popup(popup_html, parse_html=False)
    circle.add_child(popup)
    
    # Adiciona marcador com texto do preço (remova se não quiser)
    preco_popup = f"<b>R${row['preco']:.2f}</b>"
    folium.Marker(
        location=[row['lat'] - 0.01, row['long']],
        icon=folium.DivIcon(
            html=f"<div style='font-size: 10pt; color: black'>{preco_popup}</div>"
        )
    ).add_to(mapa)
    
    circle.add_to(mapa)

# Legenda da escala de cores
colormap.caption = 'Preço'
colormap.add_to(mapa)

# Exibir o mapa no notebook
display(mapa)


In [73]:
import pandas as pd
import folium
from branca.colormap import linear
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output

# Carregar os dados
#df = pd.read_csv('precos_carrefour_mapa.csv')

# Garantir que os campos estejam corretos
assert all(col in df.columns for col in ['produto', 'preco', 'lat', 'long', 'loja'])

# Lista única de produtos
lista_produtos = sorted(df['produto'].unique())

# Widget 1 – Seletor de Produto
produto_widget = widgets.Dropdown(
    options=lista_produtos,
    description='Produto:',
    layout=widgets.Layout(width='50%')
)

# Widget 2 – Checkbox para mostrar ou não os preços
mostrar_precos_widget = widgets.Checkbox(
    value=True,
    description='Exibir preços no mapa'
)

# Botão para gerar o mapa
botao_gerar = widgets.Button(description='Gerar Mapa', button_style='success')

# Output para exibir o mapa
output = widgets.Output()

# Função que gera o mapa com base nas seleções
def gerar_mapa(b):
    with output:
        clear_output()
        
        produto_selecionado = produto_widget.value
        exibir_preco = mostrar_precos_widget.value
        
        df_filtrado = df[df['produto'] == produto_selecionado]

        mapa = folium.Map(location=[-15.793889, -47.882778], zoom_start=4)
        
        preco_minimo = df_filtrado['preco'].min()
        preco_maximo = df_filtrado['preco'].max()
        colormap = linear.YlOrRd_09.scale(preco_minimo, preco_maximo)

        for _, row in df_filtrado.iterrows():
            circle = folium.CircleMarker(
                location=[row['lat'], row['long']],
                radius=10,
                color='black',
                fill=True,
                fill_opacity=0.7,
                fill_color=colormap(row['preco'])
            )
            
            popup_html = f"<b>{row['loja']}</b><br>{row['produto']}<br>Preço: <b>R${row['preco']:.2f}</b>"
            popup = folium.Popup(popup_html, parse_html=False)
            circle.add_child(popup)
            
            if exibir_preco:
                preco_popup = f"<b>R${row['preco']:.2f}</b>"
                folium.Marker(
                    location=[row['lat'] - 0.01, row['long']],
                    icon=folium.DivIcon(
                        html=f"<div style='font-size: 10pt; color: black'>{preco_popup}</div>"
                    )
                ).add_to(mapa)

            circle.add_to(mapa)

        colormap.caption = 'Preço'
        colormap.add_to(mapa)
        display(mapa)

# Conectar o botão à função
botao_gerar.on_click(gerar_mapa)

# Interface interativa
display(widgets.VBox([
    widgets.HTML(value="<h3>1. Escolha um produto</h3>"),
    produto_widget,
    widgets.HTML(value="<h3>2. Deseja visualizar os preços no mapa?</h3>"),
    mostrar_precos_widget,
    botao_gerar,
    output
]))


In [83]:
import pandas as pd
import folium
from branca.colormap import linear
from IPython.display import display, HTML, clear_output, IFrame as IFrameDisplay
import ipywidgets as widgets

# 1. Carregar os dados
#df = pd.read_csv('precos_carrefour_mapa.csv')

# Verificar se as colunas necessárias existem
colunas_necessarias = ['produto', 'preco', 'lat', 'long', 'loja']
for col in colunas_necessarias:
    assert col in df.columns, f"Coluna obrigatória ausente: {col}"

# 2. Criar lista única de produtos
lista_produtos = sorted(df['produto'].unique())

# 3. Criar widgets
produto_1_widget = widgets.Dropdown(
    options=lista_produtos,
    description='Produto 1:',
    layout=widgets.Layout(width='45%')
)

produto_2_widget = widgets.Dropdown(
    options=lista_produtos,
    description='Produto 2:',
    layout=widgets.Layout(width='45%')
)

mostrar_precos_widget = widgets.Checkbox(
    value=True,
    description='Exibir preços no mapa'
)

botao_gerar = widgets.Button(
    description='Comparar Mapas',
    button_style='info'
)

output = widgets.Output()

# 4. Função auxiliar para gerar cada mapa
def gerar_mapa_html(df_filtrado, exibir_preco):
    mapa = folium.Map(location=[-15.793889, -47.882778], zoom_start=4)

    preco_min = df_filtrado['preco'].min()
    preco_max = df_filtrado['preco'].max()
    colormap = linear.YlOrRd_09.scale(preco_min, preco_max)

    for _, row in df_filtrado.iterrows():
        circle = folium.CircleMarker(
            location=[row['lat'], row['long']],
            radius=10,
            color='black',
            fill=True,
            fill_opacity=0.7,
            fill_color=colormap(row['preco'])
        )

        popup_html = f"<b>{row['loja']}</b><br>{row['produto']}<br>Preço: <b>R${row['preco']:.2f}</b>"
        popup = folium.Popup(popup_html, parse_html=False)
        circle.add_child(popup)

        if exibir_preco:
            preco_popup = f"<b>R${row['preco']:.2f}</b>"
            folium.Marker(
                location=[row['lat'] - 0.01, row['long']],
                icon=folium.DivIcon(
                    html=f"<div style='font-size: 10pt; color: black'>{preco_popup}</div>"
                )
            ).add_to(mapa)

        circle.add_to(mapa)

    colormap.caption = 'Preço'
    colormap.add_to(mapa)

    return mapa

# 5. Função principal que gera os dois mapas
def gerar_mapas_lado_a_lado(b):
    with output:
        clear_output()

        produto1 = produto_1_widget.value
        produto2 = produto_2_widget.value
        exibir_preco = mostrar_precos_widget.value

        df1 = df[df['produto'] == produto1]
        df2 = df[df['produto'] == produto2]

        print(f"Produto 1: {produto1} → {len(df1)} registros")
        print(f"Produto 2: {produto2} → {len(df2)} registros")

        if df1.empty or df2.empty:
            print("❌ Um dos produtos não possui dados disponíveis.")
            return

        mapa1 = gerar_mapa_html(df1, exibir_preco)
        mapa2 = gerar_mapa_html(df2, exibir_preco)

        display(HTML(f"<h3>Comparação de Preços: <i>{produto1}</i> vs <i>{produto2}</i></h3>"))
        display(mapa1)
        display(mapa2)


# Conectar o botão à função
botao_gerar.on_click(gerar_mapas_lado_a_lado)

# 6. Exibir a interface interativa
display(widgets.VBox([
    widgets.HTML(value="<h3>1. Escolha dois produtos para comparar</h3>"),
    widgets.HBox([produto_1_widget, produto_2_widget]),
    widgets.HTML(value="<h3>2. Deseja visualizar os preços no mapa?</h3>"),
    mostrar_precos_widget,
    botao_gerar,
    output
]))


In [86]:
import streamlit as st
import pandas as pd
import folium
from branca.colormap import linear
from streamlit_folium import folium_static

# Carregar o dataset
#df = pd.read_csv('precos_carrefour_mapa.csv')

# Título
st.title("Comparação Geográfica de Preços")

# Produtos únicos
produtos = sorted(df['produto'].unique())

# Seleção dos dois produtos
col1, col2 = st.columns(2)
with col1:
    produto1 = st.selectbox("Produto 1", produtos)
with col2:
    produto2 = st.selectbox("Produto 2", produtos, index=1)

# Checkbox para exibir preços
exibir_preco = st.checkbox("Exibir preços ao lado dos marcadores", value=True)

# Filtrar dados
df1 = df[df['produto'] == produto1]
df2 = df[df['produto'] == produto2]

# Função para gerar o mapa
def gerar_mapa(df_filtrado, exibir_preco):
    mapa = folium.Map(location=[-15.793889, -47.882778], zoom_start=4)
    preco_min = df_filtrado['preco'].min()
    preco_max = df_filtrado['preco'].max()
    colormap = linear.YlOrRd_09.scale(preco_min, preco_max)

    for _, row in df_filtrado.iterrows():
        popup_html = f"<b>{row['loja']}</b><br>{row['produto']}<br>Preço: <b>R${row['preco']:.2f}</b>"
        popup = folium.Popup(popup_html, parse_html=False)
        marker = folium.CircleMarker(
            location=[row['lat'], row['long']],
            radius=10,
            color='black',
            fill=True,
            fill_color=colormap(row['preco']),
            fill_opacity=0.7
        )
        marker.add_child(popup)
        marker.add_to(mapa)

        if exibir_preco:
            folium.Marker(
                location=[row['lat'] - 0.01, row['long']],
                icon=folium.DivIcon(html=f"<div style='font-size: 10pt; color: black'><b>R${row['preco']:.2f}</b></div>")
            ).add_to(mapa)

    colormap.caption = 'Preço (R$)'
    colormap.add_to(mapa)
    return mapa

# Criar os mapas
st.markdown(f"### Mapa Comparativo: {produto1} vs {produto2}")
col1, col2 = st.columns(2)
with col1:
    st.markdown(f"**{produto1}**")
    mapa1 = gerar_mapa(df1, exibir_preco)
    folium_static(mapa1, width=500, height=500)

with col2:
    st.markdown(f"**{produto2}**")
    mapa2 = gerar_mapa(df2, exibir_preco)
    folium_static(mapa2, width=500, height=500)


2025-04-21 00:00:16.810 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-21 00:00:16.812 Session state does not function when running a script without `streamlit run`
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_53264/3755444398.py:67: DeprecationWarning:


folium_static is deprecated and will be removed in a future release, or
simply replaced with with st_folium which always passes
returned_objects=[] to the component.
Please try using st_folium instead, and
post an issue at https://github.com/randyzwitch/streamlit-folium/issues
if you experience issues with st_folium.


/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_53264/3755444398.py:72: DeprecationWarning:


folium_static is deprecated and will be removed in a future release, or
simply replaced with with st_folium which always passes
returned_objects=[] to the component.
Please try using st_folium instead, and
post an issue at https://git

In [85]:
pip install streamlit folium streamlit-folium branca


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
